### Parameters:
&emsp;$x$: number of assignees\
&emsp;$t$: estimated project workload for a single dev (hours)\
&emsp;$r_1$: intra-group productivity coefficient\
&emsp;$r_2$: inter-group productivity coefficient\
&emsp;$a$: Number of groups assigned to the project\
&emsp;$c$: average capability

### Functions:
&emsp;$I(x)$: Individual productivity


$$I(x)=r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c$$


&emsp;$G(x)$: Group productivity

$$G(x)=x \cdot I(x)=x \cdot r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c$$


&emsp;$T(x)$: Function of the estimated project time with x  assignees (hours)

$$T(x)=\frac{t}{G(x)}=\frac{t}{x \cdot r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c}$$


&emsp;$C(x)$: Function of the estimated project cost with $x$ assignees (in dev hours)

$$C(x)=\frac{t}{I(x)}=\frac{t}{r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c}$$


&emsp;$M(x)$: marginal benefit of increase assignees

$$M(x)=-\frac{dT}{dx}-\frac{dC}{dx}=\frac{t \cdot (x \cdot ln(r1) \cdot (1+x)+a)}{a \cdot x^2 \cdot r_1^{(\frac{x}{a}-1)} \cdot r_2^{(a-1)} \cdot c}, x \geq a$$


&emsp;$R(x)$: Threshold r1, minimum r1 required to have a non-negative margin with given number of groups and number of assignees

$$R(x)=e^{-\frac{a}{x \cdot (1+x)}}$$



In [5]:
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt
import math


def f(x, t, r1, r2, a, c):
    try:
        x = int(x)
        t = float(t)
        r1 = float(r1)
        r2 = float(r2)
        a = int(a)
        c = float(c)
    except:
        print(
            "please enter numbers,decimal will round down to nearest integer when necessary"
        )
        out = None
        df1 = None
    else:
        if x <= 0 or t <= 0 or r1 <= 0 or r2 <= 0 or a <= 0 or c <= 0:
            print("please enter non zeros")
            return None
        if r1 > 1 or r1 < 0.5:
            print("warning: r1 should between 0.5 and 1")
        if r2 > 1 or r2 < 0.5:
            print("warning: r2 should between 0.5 and 1")
        table1 = {
            "x": [],
            "a": [],
            "Individual productivity": [],
            "overall productivity": [],
            "Estimated time (100 dev hours)": [],
            "Estimated cost (100 dev hours)": [],
            "project time average rate of change": [],
            "cost average rate of change": [],
            "margin (100hours)": [],
        }
        table2 = {"x": [], "-dT/dx - dC/dx (100 hours)": []}
        table3 = {"x": [], "r1": []}
        initial = True
        last_tp = None
        last_cp = None
        for x_i in range(max(1, x - 10), x + 10):
            table1["x"].append(x_i)
            a_i = min(x_i, a)
            table1["a"].append(a_i)
            i_p = r1 ** (x_i / a_i - 1) * r2 ** (a_i - 1) * c
            table1["Individual productivity"].append(i_p)
            g_p = x_i * i_p
            table1["overall productivity"].append(g_p)
            t_p = t / g_p / 100
            table1["Estimated time (100 dev hours)"].append(t_p)
            c_p = t / i_p / 100
            table1["Estimated cost (100 dev hours)"].append(c_p)
            if initial:
                table1["project time average rate of change"].append(None)
                table1["cost average rate of change"].append(None)
                table1["margin (100hours)"].append(None)
                initial = False
            else:
                t_rc = t_p - last_tp
                table1["project time average rate of change"].append(t_rc)
                c_rc = c_p - last_cp
                table1["cost average rate of change"].append(c_rc)
                table1["margin (100hours)"].append(-t_rc - c_rc)
            last_cp = c_p
            last_tp = t_p
            if x_i >= a:
                table2["x"].append(x_i)
                table2["-dT/dx - dC/dx (100 hours)"].append(
                    t
                    * (x_i * math.log(r1) * (1 + x_i) + a)
                    / (a * x_i**2 * r1 ** (x / a - 1) * r2 ** (a - 1) * c)
                    / 100
                )
                table3["x"].append(x_i)
                table3["r1"].append(math.exp(-a / (x_i * (1 + x_i))))
        df1 = pd.DataFrame(table1)
        df2 = pd.DataFrame(table2)
        df3 = pd.DataFrame(table3)
        display(df1)
        plt.plot(
            df1["x"], df1["Individual productivity"], label="Individual productivity"
        )
        plt.plot(df1["x"], df1["overall productivity"], label="overall productivity")
        plt.plot(
            df1["x"],
            df1["Estimated time (100 dev hours)"],
            label="Estimated time (100 dev hours)",
        )
        plt.legend()
        plt.title("Graph1: Productivity")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        plt.plot(
            df1["x"],
            df1["Estimated time (100 dev hours)"],
            label="Estimated time (100 dev hours)",
        )
        plt.plot(
            df1["x"],
            df1["Estimated cost (100 dev hours)"],
            label="Estimated cost (100 dev hours)",
        )
        plt.legend()
        plt.title("Graph2: estimated workload(100 hours)")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        plt.plot(
            df1["x"],
            df1["project time average rate of change"],
            label="project time average rate of change",
        )
        plt.plot(
            df1["x"],
            df1["cost average rate of change"],
            label="cost average rate of change",
        )
        plt.plot(df1["x"], df1["margin (100hours)"], label="margin (100hours)")
        plt.legend()
        plt.title("Graph 3: Marginal benefit (average rate of change)")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        display(df2)
        plt.plot(
            df2["x"],
            df2["-dT/dx - dC/dx (100 hours)"],
            label="project time average rate of change",
        )
        plt.legend()
        plt.title("Graph4: Marginal benefit, -dT/dx - dC/dx (100 hours)")
        plt.axhline(y=0, color="k")
        plt.grid(True, which="both")
        plt.show()
        display(df3)
        plt.plot(df3["x"], df3["r1"], label="r1")
        plt.legend()
        plt.title("Graph5: threshold r1")
        plt.grid(True, which="both")
        plt.show()
    return None


# a=interact(f,x='number of assignee',t='project workload',r1='intra-group productivity coefficient',r2='inter-group productivity coefficient',a='number of groups',c='average capability')
a = interact(f, x="1", t="1600", r1="0.8", r2="0.8", a="2", c="1")


interactive(children=(Text(value='1', description='x'), Text(value='1600', description='t'), Text(value='0.8',…

### Regression approach to estimate r1/r2:
If we apply natural logarithm to both side of $T(x)$, and shuffle the terms we get the following equation:

$$ln(\frac{T(x)\cdot x \cdot c}{t})=-ln(r_1) \cdot (\frac{x}{a}-1)-ln(r_2) \cdot (a-1) \qquad \qquad (1)$$

Let $T=[\tau_1,\tau_2,...,\tau_k]'$ be observed project times, $\mathbf{y}=[y_1,y_0,...,y_k]'$ where $y_i=ln(\frac{\tau_i+x_i+c_i}{t_i})$, $\beta=[-ln(r_1),-ln(r_2)]'$ where $r_1 \in (0,1],\quad r_2 \in (0,1]$ and

$$
\mathbf{X}_{k,2} = 
 \begin{pmatrix}
  \frac{x_1}{a_1}-1 & a_1-1\\
  \frac{x_2}{a_2}-1 & a_2-1\\
  \vdots  & \vdots\\
  \frac{x_k}{a_k}-1 & a_k-1 
 \end{pmatrix}
$$

The quation (1) then can be rewriten as a matrix formulation of a regression model

$$\mathbf{y}=\mathbf{X}\beta+\varepsilon \qquad \qquad (2)$$

We can estimate $\beta$ by solveing the non-negative least squares

$$\hat{\beta}=\operatorname*{arg\,max}_\beta\|\mathbf{X}\beta-\mathbf{y}\|,\quad \beta\geq\mathbf{0} \qquad \qquad (3)$$



In [5]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import math
import pandas as pd

sample_size = 100

t = np.random.randint(500, 1000, sample_size)
x = np.random.randint(2, 10, sample_size)
a = [np.random.randint(1, x[i]) for i in range(sample_size)]

# target coefficents
true_coeff = [-math.log(0.9), -math.log(0.8)]
X = np.asarray([[x[i] / a[i] - 1, a[i] - 1] for i in range(sample_size)])

y_true = np.dot(X, true_coeff)

c = 1 + np.random.uniform(-0.2, 0.2, size=(sample_size,))

tau = np.exp(y_true + np.log(t) - np.log(x) - np.log(c)) + 20 * np.random.normal(
    size=(sample_size,)
)

y = np.log(tau) + np.log(x) + np.log(c) - np.log(t)

df1 = pd.DataFrame(
    {
        "true y": y_true,
        "noisy y": y,
        "x": x,
        "tau": tau,
        "t": t,
        "a": a,
    }
)
display(df1)



,true y,noisy y,x,tau,t,a
0,0.586768,0.498244,7,178.051863,807,3
1,0.621888,0.605956,8,190.225323,858,3
2,0.210721,0.370338,3,456.406871,869,1
3,0.657008,0.547381,9,189.184020,788,3
4,1.150838,1.252982,8,326.053045,769,6
...,...,...,...,...,...,...
95,0.632163,0.694931,7,287.108371,962,1
96,0.481407,0.366129,4,368.475399,888,3
97,0.539225,0.582596,8,251.791056,956,2
98,1.168398,1.265292,9,257.073640,656,6


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


reg_nnls = LinearRegression(positive=True)
reg_mode = reg_nnls.fit(X_train, y_train)

r1_ln, r2_ln = reg_nnls.coef_
r1 = math.exp(-r1_ln)
r2 = math.exp(-r2_ln)
print("projected r1,r2:", r1, r2)


y_pred = reg_mode.predict(X_test)
r2_score_nnls = r2_score(y_test, y_pred)
print("r2_score:", r2_score_nnls)


projected r1,r2: 0.8987925907502036 0.7936446740617683
r2_score: 0.9662196777533975
